<!--NAVIGATION-->

<a href="https://colab.research.google.com/github/bpesquet/machine-learning-katas/blob/master/notebooks/katas/algorithms/KNN_BreastCancer.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open in Google Colaboratory"></a>


## Instructions

This is a self-correcting exercise generated by [nbgrader](https://github.com/jupyter/nbgrader). 

Fill in any place that says `YOUR CODE HERE` or `YOUR ANSWER HERE`. Run subsequent cells to check your code.

---

# Kata: Diagnose Breast Tumors with K-Nearest Neighbors

In this kata, you'll use a K-Nearest Neighbors classifier to help diagnose breast tumors.

The [Breast Cancer][1] dataset is used for multivariate binary classification between benign and maligant tumors. There are 569 total samples with 30 features each. Features were computed from a digitized image of a fine needle aspirate of a breast mass. They describe characteristics of the cell nuclei present in the image.

![](images/breast-cancer-logo.jpg)

[1]: https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+(Diagnostic)

## Package setup

In [1]:
# Import base packages
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [2]:
# Setup plots
%matplotlib inline
plt.rcParams['figure.figsize'] = 10, 8
%config InlineBackend.figure_format = 'retina'
sns.set()

### Question

Import the needed packages.

In [35]:
# Import ML packages (edit this list if needed)
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler

## Step 1: Loading the data

In [4]:
dataset = load_breast_cancer()

# Put data in a pandas DataFrame
df_breast_cancer = pd.DataFrame(dataset.data, columns=dataset.feature_names)
# Add target and class to DataFrame
df_breast_cancer['target'] = dataset.target
df_breast_cancer['class'] = dataset.target_names[dataset.target]
# Show 10 random samples
df_breast_cancer.sample(n=10)

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target,class
407,12.850,21.37,82.63,514.5,0.07551,0.08316,0.06126,0.01867,0.1580,0.06114,...,91.63,645.8,0.09402,0.1936,0.18380,0.05601,0.2488,0.08151,1,benign
251,11.500,18.45,73.28,407.4,0.09345,0.05991,0.02638,0.02069,0.1834,0.05934,...,83.12,508.9,0.11830,0.1049,0.08105,0.06544,0.2740,0.06487,1,benign
27,18.610,20.25,122.10,1094.0,0.09440,0.10660,0.14900,0.07731,0.1697,0.05699,...,139.90,1403.0,0.13380,0.2117,0.34460,0.14900,0.2341,0.07421,0,malignant
228,12.620,23.97,81.35,496.4,0.07903,0.07529,0.05438,0.02036,0.1514,0.06019,...,90.67,624.0,0.12270,0.3454,0.39110,0.11800,0.2826,0.09585,1,benign
427,10.800,21.98,68.79,359.9,0.08801,0.05743,0.03614,0.01404,0.2016,0.05977,...,83.69,489.5,0.13030,0.1696,0.19270,0.07485,0.2965,0.07662,1,benign
341,9.606,16.84,61.64,280.5,0.08481,0.09228,0.08422,0.02292,0.2036,0.07125,...,71.25,353.6,0.12330,0.3416,0.43410,0.08120,0.2982,0.09825,1,benign
247,12.890,14.11,84.95,512.2,0.08760,0.13460,0.13740,0.03980,0.1596,0.06409,...,105.00,639.1,0.12540,0.5849,0.77270,0.15610,0.2639,0.11780,1,benign
281,11.740,14.02,74.24,427.3,0.07813,0.04340,0.02245,0.02763,0.2101,0.06113,...,84.70,533.7,0.10360,0.0850,0.06735,0.08290,0.3101,0.06688,1,benign
252,19.730,19.82,130.70,1206.0,0.10620,0.18490,0.24170,0.09740,0.1733,0.06697,...,159.80,1933.0,0.17100,0.5955,0.84890,0.25070,0.2749,0.12970,0,malignant
269,10.710,20.39,69.50,344.9,0.10820,0.12890,0.08448,0.02867,0.1668,0.06862,...,76.51,410.4,0.13350,0.2550,0.25340,0.08600,0.2605,0.08701,1,benign


## Step 2: Preparing the data

### Question

Compute the number of features of the dataset into the `num_features` variable.

In [14]:
num_features = len(dataset.feature_names)

In [15]:
print(f'Number of features: {num_features}')

assert num_features == 30

Number of features: 30


### Question

In order to evaluate class distribution, compute the number of benign and malignant tumors into the `num_benign` and `num_malignant` variables respectively.

In [29]:
num_benign = len(df_breast_cancer[df_breast_cancer['class'] == 'benign'])
num_malignant = len(df_breast_cancer[df_breast_cancer['class'] == 'malignant'])

In [30]:
print(f'Benign count: {num_benign}. Malignant count: {num_malignant}')

assert num_benign == 357
assert num_malignant == 212

Benign count: 357. Malignant count: 212


In [31]:
# Store input and labels
x = dataset.data
y = dataset.target

print(f'x: {x.shape}. y: {y.shape}')

x: (569, 30). y: (569,)


### Question

Split the dataset into training and test sets with a 25% ratio. Use variables `x_train`, `y_train`, `x_test` and `y_test`.

In [45]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.25)

In [46]:
print(f'x_train: {x_train.shape}. y_train: {y_train.shape}')
print(f'x_test: {x_test.shape}. y_test: {y_test.shape}')

assert x_train.shape == (426, 30)
assert y_train.shape == (426, )
assert x_test.shape == (143, 30)
assert y_test.shape == (143,)

x_train: (426, 30). y_train: (426,)
x_test: (143, 30). y_test: (143,)


### Question

Scale features by standardization while preventing information leakage from the test set.

In [48]:
scaler = StandardScaler().fit(x_train)
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)


In [49]:
mean_train = x_train.mean()
std_train = x_train.std()
print(f'mean_train: {mean_train}. std_train: {std_train}')

assert np.abs(np.max(mean_train)) < 10**-6
assert np.abs(np.max(std_train - 1)) < 10**-6

mean_train: -4.04753608476802e-16. std_train: 1.0


## Step 3: Creating a classifier

### Question

Create a `KNeighborsClassifier` instance using only one nearest neighbor, store it into the `model` variable, and fit the training data.

In [76]:
model = KNeighborsClassifier(n_neighbors=1)
model.fit(x_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=1, p=2,
                     weights='uniform')

## Step 4: Evaluating the classifier

In [74]:
# Compute accuracy on training and test sets
train_acc = model.score(x_train, y_train)
test_acc = model.score(x_test, y_test)

print(f'Training accuracy: {train_acc * 100:.2f}%')
print(f'Test accuracy: {test_acc * 100:.2f}%')

Training accuracy: 100.00%
Test accuracy: 94.41%


### Question

Display precision, recall and f1-score for the classifier on test data. Interpret the results.

'minkowski'

### Question

Go back to step 3 and try to find the best value for the `k` number of nearest neighbors.